# L'algoritmo CYK

## La tabella, sua relazione con l'input

In [1]:
INPUT = 'atest'

R = {}
for l in range(len(INPUT), 0, -1):
    for i in range(1, len(INPUT) - l + 2): 
        R[(i, l)] = INPUT[i - 1 : i + l - 1]

R

{(1, 5): 'atest',
 (1, 4): 'ates',
 (2, 4): 'test',
 (1, 3): 'ate',
 (2, 3): 'tes',
 (3, 3): 'est',
 (1, 2): 'at',
 (2, 2): 'te',
 (3, 2): 'es',
 (4, 2): 'st',
 (1, 1): 'a',
 (2, 1): 't',
 (3, 1): 'e',
 (4, 1): 's',
 (5, 1): 't'}

In [2]:
from liblet import cyk2table

cyk2table(R)

### Riempimento offline

In [3]:
def offline(fill, n):
    R = {}
    for l in range(1, n + 1):
        for i in range(1, n - l + 2): 
            R[(i, l)] = fill(R, i, l)
    return R

In [4]:
def fill_with_nij(R, i, j):
    global n
    n += 1
    return '{} @ ({}, {})'.format(n, i, j)

In [5]:
n = 0
cyk2table(offline(fill_with_nij, 5))

### Riempimento online

In [6]:
def online(fill, n):
    R = {}
    for d in range(1, n + 1):
        for i in range(d, 0, -1):
            R[(i, d - i + 1)] = fill(R, i, d - i + 1)
    return R

In [7]:
n = 0
cyk2table(online(fill_with_nij, 5))

## Riempimento usando G (CNF) e input

In [8]:
def cyk_fill(G, INPUT):
    def fill(R, i, l):
        res = set()
        if l == 1:
            for (A, (a, *α)) in G.P: 
                if not α and a == INPUT[i - 1]:
                    res.add(A)
        else:
            for k in range(1, l):
                for A, α in G.P:
                    if len(α) != 2: continue
                    B, C = α
                    if B in R[(i, k)] and C in R[(i + k, l - k)]:
                        res.add(A)
        return res
    return fill

In [9]:
from liblet import Grammar

G = Grammar.from_string("""
S -> A S
A -> a
S -> .
""")

In [10]:
INPUT = 'aaa.'

R = online(cyk_fill(G, INPUT), len(INPUT))
cyk2table(R)

### Un esempio dal libro di testo

In [11]:
# fig. 4.15, pag. 123 

G = Grammar.from_string("""
Number -> 0|1|2|3|4|5|6|7|8|9 
Number -> Integer Digit
Number -> N1 Scale' | Integer Fraction
N1 -> Integer Fraction
Integer -> 0|1|2|3|4|5|6|7|8|9 
Integer -> Integer Digit
Fraction -> T1 Integer
T1 -> .
Scale' -> N2 Integer
N2 -> T2 Sign
T2 -> e
Digit -> 0|1|2|3|4|5|6|7|8|9 
Sign -> + | -
""")
G

Grammar(N={Digit, Fraction, Integer, N1, N2, Number, Scale', Sign, T1, T2}, T={+, -, ., 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, e}, P=(Number -> 0, Number -> 1, Number -> 2, Number -> 3, Number -> 4, Number -> 5, Number -> 6, Number -> 7, Number -> 8, Number -> 9, Number -> Integer Digit, Number -> N1 Scale', Number -> Integer Fraction, N1 -> Integer Fraction, Integer -> 0, Integer -> 1, Integer -> 2, Integer -> 3, Integer -> 4, Integer -> 5, Integer -> 6, Integer -> 7, Integer -> 8, Integer -> 9, Integer -> Integer Digit, Fraction -> T1 Integer, T1 -> ., Scale' -> N2 Integer, N2 -> T2 Sign, T2 -> e, Digit -> 0, Digit -> 1, Digit -> 2, Digit -> 3, Digit -> 4, Digit -> 5, Digit -> 6, Digit -> 7, Digit -> 8, Digit -> 9, Sign -> +, Sign -> -), S=Number)

In [12]:
INPUT = '32.5e+1'

R = offline(cyk_fill(G, INPUT), len(INPUT))
cyk2table(R)